In [29]:
from UDRead import *
from parse_asd import *

parity = "even"
order = "E"

# Atom = "Ca I"
# ri = True
# nist_max = 51 if parity=="odd" else 51 # Ca I

# Atom = "Sr I"
# ri = False
# nist_max = 50 if parity== "odd" else 50 # Sr I

Atom = "In II"
ri = False
nist_max = 36 if parity== "odd" else 32 # Sr I

# Atom = "In I"
# ri = False
# nist_max = 28 if parity== "odd" else 27 # Sr I

# Atom = "Y II"
# ri = False
# nist_max = 43 if parity== "odd" else 40 # Sr I

# Atom = "Cs VII"
# ri = True # fac=2 for Sr, fac=4 for Ca
# nist_max = 32 if parity== "odd" else 31 # Sr I

Name = Atom.replace(" ","_")

# # DATA NIST
# Store Filtered data of even or odd parity in DATA_Filtered/NIST/ 
url_nist = generate_asd_url(Atom,parity)
# print(url_nist)
data_nist = generate_df_from_asd(url_nist)

data_nist = reformat_df_to_atomdb(data_nist)
df_to_csv(data_nist,"DATA_Filtered/NIST/"+Atom,parity,ri=ri)

## DATA UD
# Sorted formatting, Converted .RES to .csv and stored Filtered data of even or odd parity in DATA_Filtered/UD/ 
convert_res_to_csv("DATA_RAW/UD/"+Name+"_UD_"+parity.capitalize()+".RES")

path_nist = "DATA_Filtered/NIST/"+Name+"_NIST_"+parity.capitalize()+".csv"
path_ud = "DATA_Filtered/UD/"+Name+"_UD_"+parity.capitalize()+".csv"

df_nist,df_ud,ref_E = Dataframe(path_nist,path_ud,nist_max)

# data_final = MainCode(path_nist, path_ud,nist_max,fac,parity,Ordering=order,Same=same)

# # Export
# path = "DATA_Output/"+Name+"_"+parity.capitalize()+".txt" #specify path for export
# ConvertToTXT(data_final,path)

In [30]:
new_config = Corrected_Config(df_ud,df_nist,ManualCorrection=False)

In [31]:
Ordering=order


df_nist,df_ud,ref_E = Dataframe(path_nist,path_ud,nist_max)

data_csv = []
ao_used = []
roundd = 3

ref_E = 0
for i in range(nist_max):
    config_nist, term_nist, j_nist, level_nist, uncer_nist = Data_Nist(i,df_nist)
    data_nist = [config_nist, term_nist, j_nist, round(level_nist+ref_E,roundd),uncer_nist]

    nist_used = 0

    jth,jf,de = FindJth(i,df_nist,df_ud,corr_config=new_config,which_j=True)
#     if jf==2 and (1 in IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config))==True:jth=-1 # avoiding duplicates
    if jth!=-1:
        ao_used.append(jth)
        nist_used = 1
        config1_ao, config2_ao, term_ao, j_ao, level_ao,level_au, per1,per2,uncer_ud = Data_UD(jth,df_ud)
        final_config = config2_ao if jf==2 else config1_ao # final config
        Ediff = round(abs(level_nist-level_ao),1)
        Eperc = round(100*Ediff/level_nist,2) if level_nist!=0 else 0
        EpercStr = str(Eperc)+"%"
        if config2_ao=='': config2_ao='-'
        data_csv.append(data_nist+["-",new_config[jth],config1_ao,config2_ao, term_ao, j_ao, round(level_ao+ref_E,roundd),uncer_ud,level_au,Ediff,EpercStr, per1,per2])


    ## remaining nist states
    if nist_used==0:
        data_csv.append([config_nist, term_nist, j_nist, round(level_nist+ref_E,roundd),uncer_nist,"-","-","-","-","-","-",round(level_nist+ref_E,roundd),"-","-","-","-","","-"])

## remaining ao states
for j in range(len(df_ud)):
    config1_ao, config2_ao, term_ao, j_ao, level_ao,level_au, per1,per2,uncer_ud = Data_UD(j,df_ud)
    if config2_ao=='': config2_ao='-'
    if (j in ao_used)==False:
        data_csv.append(["-","-","-",round(level_ao+ref_E,roundd),"-","-",new_config[j],config1_ao, config2_ao, term_ao, j_ao, round(level_ao+ref_E,roundd),uncer_ud,level_au,"-","-", per1,per2])


header = [" Config","  Term","  J","  Level (cm⁻¹)","Uncertainty (cm-1)","Final Config","   OrgConfig1","  CorConfig1","  Config2","  Term","  J","  Level (cm⁻¹)","  Uncertainty (cm⁻¹)","  Level (au)","    \u0394E","    \u0394E%","  I","  II"]
data = np.array(data_csv)

# sort data
sort = data.T[3].astype(float).argsort()
data = data[sort]

for i in range(len(data)):
    if data[i][0]=="-": data[i][3] = "-"
    if data[i][0]=="": data[i][3] = ""

for i in range(len(data)):# 6 and 10 neends to change if added any new column between them
    if data[i][6]=="-": data[i][11] = "-"
    if data[i][6]=="": data[i][11] = ""

print(np.where(np.bincount(ao_used) > 1)[0]) # duplicates

if Ordering=="E": data_final = FinalSortE(data)
# if Ordering=="T": data_final = FinalSortT(data)

[]


In [32]:
data_final=data

# remove = [-1,-2]
# data_new = np.delete(data_final[1], remove, axis=0)
# header_new = np.delete(data_final[0], remove, axis=0)

In [33]:
pd.DataFrame(data,columns=header)

,Config,Term,J,Level (cm⁻¹),Uncertainty (cm-1),Final Config,OrgConfig1,CorConfig1,Config2,Term,J,Level (cm⁻¹),Uncertainty (cm⁻¹),Level (au),ΔE,ΔE%,I,II
0,5s2,1S,0,0,0,-,5s2,5s2,-,1S,0,0.0,,1.63408613,0.0,0%,94.0%,
1,5s 6s,3S,1,93923.884,0.005,-,5s 6s,5s 6s,-,3S,1,86445.6,,1.24021078,7478.3,7.96%,93.3%,
2,5s 6s,1S,0,97030.212,0.012,-,5s 6s,5s 6s,-,1S,0,89898.2,,1.22447965,7132.0,7.35%,86.2%,
3,5s 5d,1D,2,97628.436,0.009,-,5s 5d,5s 5d,5p2,1D,2,88268.0,,1.23190737,9360.4,9.59%,57.0%,40.1%
4,5p2,3P,0,101608.06,0.2,-,5p2,5p2,-,3P,0,95249.0,,1.20009979,6359.1,6.26%,96.7%,
5,5s 5d,3D,1,102088.72,0.04,-,5s 5d,5s 5d,-,3D,1,94251.4,,1.20464511,7837.3,7.68%,89.1%,
6,5s 5d,3D,2,102174.69,0.03,-,5s 5d,5s 5d,-,3D,2,94323.9,,1.20431484,7850.8,7.68%,89.2%,
7,5s 5d,3D,3,102308.397,0.02,-,5s 5d,5s 5d,-,3D,3,94438.8,,1.20379129,7869.6,7.69%,89.2%,
8,5p2,3P,1,103249.39,0.19,-,5p2,5p2,-,3P,1,96593.9,,1.19397204,6655.5,6.45%,96.9%,
9,5p2,3P,2,105565.283,0.012,-,5p2,5p2,-,3P,2,98593.2,,1.18486262,6972.1,6.6%,94.9%,


In [23]:
# 100*(47242.2-47040.007)/47040.007,100*(47242.2-47466.014)/47466.014

In [24]:
# IsBeingUsed(49,df_nist,df_ud,corr_config=new_config,OutAll=True)
# IsBeingUsed(49,df_nist,df_ud,OutAll=True)

In [9]:
# StartingConfigs(df_nist)

In [58]:
new_config = np.full(len(df_ud),"",dtype=object)
list_config,list_term,list_number,count = StartingConfigs(df_nist)

for j in range(29):
    nf,jf,de = IsBeingUsed(j,df_nist,df_ud)
    config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
    mul = int(term_ao[0]) if jf==0 else int(df_nist[nf][1][0])
#     print(j,jf)
#     mul = int(df_nist[nf][1][0]) if 
#     print(j, mul)
#     print(j,IsBeingUsed(j,df_nist,df_ud,OutAll=True),jf)
    if jf==0 or jf==1: config = config1_ao
    if jf==2:
        config = config1_ao
        new_config_check = Correct_Config(j,config,mul,new_config,list_config,list_term,list_number,count,Final=False)
        nf1,jf1,de1 = IsBeingUsed(j,df_nist,df_ud,corr_config=new_config_check)

        config = config2_ao
#         new_config_check = Correct_Config(j,config,mul,new_config,list_config,list_term,list_number,count,Final=False)
        new_config_check[j] = config2_ao
#         print(config2_ao,new_config_check[j])
        nf2,jf2,de2 = IsBeingUsed(j,df_nist,df_ud,corr_config=new_config_check)
        
        if de1!=0 and de2==0: config=config1_ao
        elif de1==0 and de2!=0: config=config2_ao
        elif de1!=0 and de2!=0: config=config1_ao if de1<=de2 else config2_ao
        else: config=config1_ao

    new_config,list_config,list_term,list_number,count = Correct_Config(j,config,mul,new_config,list_config,list_term,list_number,count,Final=True)
#     print(j,config,new_config[j],config1_ao,config2_ao,term_ao,mul)
#     print(j,list_number)
    print(j,new_config[j],"   ",config1_ao,"   ",config2_ao,"   ",term_ao,"   ",nf1,nf2,mul,de1,de2)
    
#     print(j,IsBeingUsed(j,df_nist,df_ud,corr_config=new_config,OutAll=True))

0 4.0 5 0 -1.0 ['5', 's2', '5', 'p']
0 5s2 5p     5s2 5p          2P     24 0 2 27.55 0
1 4.0 5 0 -1.0 ['5', 's2', '5', 'p']
1 5s2 5p     5s2 5p          2P     24 0 2 27.55 0
2 5.0 7 1 -2.0 ['5', 's2', '7', 'p']
2 5.0 7 1 -2.0 ['5', 's2', '7', 'p']
2 5s2 6p     5s2 7p     5s2 6p     2P     2 2 2 1.59 1.59
3 5.0 7 1 -2.0 ['5', 's2', '7', 'p']
3 5.0 7 1 -2.0 ['5', 's2', '7', 'p']
3 5s2 6p     5s2 7p     5s2 6p     2P     3 3 2 1.56 1.56
4 6.0 8 1 -2.0 ['5', 's2', '8', 'p']
4 5s2 7p     5s2 8p     5s2 9p     2P     3 3 2 1.56 1.56
5 6.0 8 1 -2.0 ['5', 's2', '8', 'p']
5 5s2 7p     5s2 8p     5s2 9p     2P     3 3 2 1.56 1.56
6 3.0 6 2 -3.0 ['5', 's2', '6', 'f']
6 5s2 4f     5s2 6f     5s2 5f     2F     3 3 2 1.56 1.56
7 3.0 5 1 -2.0 ['5', 's2', '5', 'f']
7 5s2 4f     5s2 5f     5s2 6f     2F     3 3 2 1.56 1.56
8 7.0 10 2 -3.0 ['5', 's2', '10', 'p']
8 5s2 8p     5s2 10p     5s2 11p     2P     3 3 2 1.56 1.56
9 7.0 10 2 -3.0 ['5', 's2', '10', 'p']
9 5s2 8p     5s2 10p     5s2 11p     2P   

In [3]:
12,int(abs(9.0-12)-1)

(12, 2)

In [7]:
Sep_Config('5s2 12f')

['5', 's2', '12', 'f']

In [45]:
list_term,"\n",list_number,list_config

(array([2., 2.]),
 '\n',
 array([11.,  9.]),
 array(['5s2 1p', '5s2 1f'], dtype='<U32'))

In [21]:
new_config[28]='5s 10f'

In [56]:
jth=28
# IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config,OutAll=True)
IsBeingUsed(jth,df_nist,df_ud,OutAll=True)

([7], [2], [46.37])

In [48]:
# FindJth(7,df_nist,df_ud,corr_config=new_config,which_j=True)
FindJthAll(7,df_nist,df_ud)

([], [28, 30], [], [46.37, 64.71])

In [5]:
jth=27
new_config[jth],df_ud[jth]

('5s 4f',
 array([28, '5s 7f', '1F3', 1.03772812, 93856.4, 0.2, 3, 3, '1.01',
        '70.7%', '', '', '', ''], dtype=object))

In [33]:
df_nist[6]

array(['5s2 4f', '2F', 2.5, 39707.59, 0.03, False], dtype=object)

In [65]:
# jth,jf,dE = FindJth(i,df_nist,df_ud,corr_config=new_config,which_j=True)
# print(jth,jf,dE)
# print(new_config[jth])
# IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config,OutAll=True)
# # IsBeingUsed(jth,df_nist,df_ud)

In [48]:
j=25

config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
config1_ao,config2_ao,new_config[j]

('4s 8s', '4s 7s', '4s 7s')

,0,1,2,3,4,5
0,NaN,1S,0,0.0,NaN,False
1,5s 4d,3D,1,18159.04,0.003,False
2,5s 4d,3D,2,18218.784,0.003,False
3,5s 4d,3D,3,18319.261,0.004,False
4,5s 4d,1D,2,20149.685,0.003,False
5,5s 6s,3S,1,29038.773,0.004,False
6,5s 6s,1S,0,30591.825,0.016,False
7,5s 5d,1D,2,34727.447,0.005,False
8,5s 5d,3D,1,35006.908,0.007,False
9,5s 5d,3D,2,35021.989,0.012,False


In [5]:
new_config = np.full(len(df_ud),"",dtype=object)
list_config,list_term,list_number,count = StartingConfigs(df_nist)

In [6]:
conf=Sep_Config(config)
dummy_config = Dummy_Config(config)
idx_dummy = np.where((list_config.astype(str)==dummy_config) & (list_term.astype(int)==mul))[0]

n = int(conf[-2])
# Store term and config for comparison
if len(idx_dummy)==0:
    list_config=np.append(list_config,dummy_config)
    list_term=np.append(list_term,mul)
    list_number=np.append(list_number,n)
    count = np.append(count,0)
    
# main correction part
idx_dummy = np.where((list_config.astype(str)==dummy_config) & (list_term.astype(int)==mul))[0] # index of dummy config present in list_config
if conf[-1]=='s':mul=1
n_prev = list_number[idx_dummy[0]]
if abs(n_prev-n)>1:
    n=n-int(abs(n_prev-n)-1)


conf[-2]=str(n)
new_config[jth]=Combine_Config(conf)


if count[idx_dummy[0]]!=mul: count[idx_dummy[0]]+=1
# re-initian for the next loop
if count[idx_dummy[0]]==mul:
    list_number[idx_dummy[0]]=n
    count[idx_dummy[0]]=0
    
mul,count,new_config,list_number,list_config

(2,
 array([0., 1., 0.]),
 array(['', '', '', '', '5s2 5d', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', ''], dtype=object),
 array([5., 4., 4.]),
 array(['1s2 1s', '1s2 1d', '1s 1p2'], dtype='<U32'))

In [22]:
jth,jf,dE = FindJth(i,df_nist,df_ud,corr_config=new_config,which_j=True)
print(jth,jf,dE)
print(new_config[jth])
# IsBeingUsed(jth,df_nist,df_ud,corr_config=new_config,OutAll=True)
IsBeingUsed(jth,df_nist,df_ud)

4 1 37.81
5s2 5d


(7, 1, 20.0)

In [10]:
j=6

config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(j,df_ud)
config1_ao, config2_ao, term_ao, j_ao

('5s2 8s', '5s2 7s', '2S', 0.5)

In [12]:
# StartingConfigs(df_nist)

In [6]:
i=10
FindJthAll(i,df_nist,df_ud)

([8], [], [2.37], [])

In [36]:
# df_nist[i]

In [35]:
# OrgConfig1,config1_ao, config2_ao, term_ao, j_ao, level_ao,Levelau, per1,per2,uncer_ud = Data_UD(i,df_ud)
# OrgConfig1,config1_ao, config2_ao, term_ao, j_ao

''